# Building a Better Isolation Evaluation Function
#### Hoxie Ackerman (hoxiea@gmail.com)

## 1: Introduction

After implementing Minimax search and Iterative-Deepening Search with Alpha-Beta pruning for my IsolationPlayers, I next tackled the problem of developing an Isolation evaluation function that outperforms the provided  `improved` evaluation function. 

For reference, the `improved` evaluation function for a certain board configuration is:

In [5]:
def improved_score(game, player):
    """
    The "Improved" evaluation function discussed in lecture that outputs a
    score equal to the difference in the number of moves available to the
    two players.
    """
    if game.is_loser(player):
        return float("-inf")
    if game.is_winner(player):
        return float("inf")
    own_moves = len(game.get_legal_moves(player))
    opp_moves = len(game.get_legal_moves(game.get_opponent(player)))
    return float(own_moves - opp_moves)

To refresh our memories, the **evaluation function** is the method that an IsolationPlayer uses to score Boards that it encounters in its search tree. A useful evaluation function will consider various properties of the Board from our player's perspective, and should consistently assign higher scores to Boards in which we have a better chance of winning the game.

`improved_score` does something very reasonable along these lines: Boards where we have more legal moves will score higher, as will Boards where our opponent has fewer legal moves. These are extremely reasonable Board properties to consider in a game where you lose by running out of legal moves.

As Dr. Starner mentioned in Lecture 9 of the "Project: Build a Game-Playing Agent" videos, one can consider adjust the "weights" of the terms in the linear combination of `improved_score`. In particular, he proposed a modified version of `improved_score` that would essentially return `own_moves - 2 * opp_moves`. He claimed that IsolationPlayers using this evaluation function would, on average, play more aggresively, leaning more towards Boards where the opponent has fewer moves, which is ultimately how you win in this game. So that seems like a very reasonable modification to make.

But is "-2" really the optimal coefficient for `opp_moves`? And are there other features of a Board, aside from `own_moves` and `opp_moves` that would also point our IsolationPlayers towards better board positions? Let's investigate.

## 2: How Will We Know When We're Done?

If the goal is to construct an evaluation function that outperforms `score_improved`, I think it's important to define what we mean by "outperform." Given a new-and-possibly-improved evaluation function `contender`, it seems like I have two options for demonstrating superiority:

1. Show, using the tools of math and logic, that `contender` is provably better than score_improved.
2. Show empirically that `contender` performs better, on average, than `score_improved`.

Option 2 seems much easier, so let's pursue this approach.

One reason why Option 2 is easier: the project comes with `tournament.py`, which pits the **test agents** `AB_Improved` (an `AlphaBetaPlayer` using `score_improved`) and one or more `AlphaBetaPlayers` equipped custom evaluation functions against a motley gauntlet of seven opponents: an opponent that moves randomly, three `MinimaxPlayer`s using various evaluation functions, and three `AlphaBetaPlayer`s using various evaluation functions. Overall Win Rates are calculated for each test agent.

So that's the plan. Let's see an example of the plan in action:

In particular, suppose I had developed nine different evaluation functions using the following pseudocode algorithm:
```
for own_moves_weight in (1, 2, 3):
    for opp_moves_weight in (-3, -2, -1):
        modify improved_score to use own_moves_weight and opp_moves_weight
```

According to the plan, I should equip some `AlphaBetaPlayers` with these evaluation functions and run some tournaments. I did this, and obtained the following output, where the tuple in the test_agent's name indicates the weights used:

![Effects of varying opp_moves_weight, with constant own_moves_weight = 1.](report-pics/section2/1-DONE.png)

![Effects of varying opp_moves_weight, with constant own_moves_weight = 2.](report-pics/section2/2-DONE.png)

![Effects of varying opp_moves_weight, with constant own_moves_weight = 3.](report-pics/section2/3-DONE.png)

So let's see... (1, -1) actually seems like the best combination tested, with a Win Rate of 75.7%. But (3, -2) was also pretty potent, with a Win Rate of 74.3%. And in all three runs, at least one of these weighted test agents outperformed AB_Improved. So give or take some fine-tuning, we've got our improved evaluation functions, right?

Maybe, but I'm not convinced, and you shouldn't be either. This is a random process (for example: `AB_Improved` won 67.1% of its games in the first and third tournaments, and only 61.4% in the second tournament). If I were to repeat this entire process with three more tournaments, the win rates could change. How can we be sure that what we're seeing in a given run is signal (an actual difference in performance between evaluation functions) and not noise (random variation)?

This is, of course, a statistics problem. The randomness makes it almost impossible to say anything with certainty. But a statistical test would (ideally) let us conclude with high confidence/probability that the true win rate for one of our custom evaluation functions was higher than it was for `score_improved.`

I presented this point of view in the [Udacity forums](https://discussions.udacity.com/t/sources-of-variation-in-tournaments-and-why-forfeits/268387/5?u=hha) (that this was inherently a statistical problem, and that we should be thinking about statistical tests), and it was mostly glossed over. The proposed solution to this problem was just to run more matches (i.e. increase the sample size to decrease noise levels for the means), and just eyeball the results for differences.

Even if we're not going to perform a formal statistical test, it still makes sense to get a feel for the amount of variability in this process. If `AB_Improved` is always between 61.4% and 67.1% (the values we observed in the three tournaments above), for example, then a `contender` with win rates consistently in the 70+% range would seem to be outperforming `AB_Improved` with high probability.

As it turns out, with 5 fair matches per matchup (i.e. 10 games per matchup, as above), there's siginificantly more variability than 61-67%. **Confession time: I didn't actually implement those nine evaluation functions. Every single one of the test agents in the above three tournaments was an `AB_Improved` agent.** So based on those results, we see that with 5 fair matches per matchup, the quantity that we're supposed to be comparing our performance against can be anywhere between about 60% and about 75%. Maybe there's even greater variability that we didn't happen to see in those three runs.

That seems like quite a bit of background noise to me. And the more I thought about this, the stranger that seemed to me. After all, this is a fully observable, deterministic, static, discrete environment that we're playing in here. And the algorithms that our AIs are using (aside from the `Random` opponent, who just selects a random legal move on each turn) are almost completely deterministic. Where is all this variation coming from?

## 3: Analysis of Variance

First, let's make sure that we can really reduce the variability in Win Rates by increasing the number of fair matches (let's call that `NUM_MATCHES`). Here's a tournament with all four test agents using `score_improved` and `NUM_MATCHES` increased to 25:

![Tournament, all test agents using ab_improved, NUM_MATCHES = 25](report-pics/section3/tourn1.png)

So indeed, by playing more matches per matchup, the Win Rates are within just a couple percentage points of about 67%, which is the figure that's often reported in the forums for working code.

The problem with this approach is that it's computationally expensive. A gridsearch in a multidimensional weight space to find good weights for the various board features in our custom evaluation function(s) could take hours or days. If we need to pay that price, then that's life, but consider this row from that output for a second:

![Same evaluation function for all test agents, same starting Board configurations, same opponent, mostly deterministic search algorithms: where is that variation coming from?](report-pics/section3/tourn1-row.png)

Why is there so much variation in those outcomes? As I said above, this is a fully observable, deterministic, static, discrete environment, and our agents are using what appear to be mostly deterministic search algorithms to select their moves. 

Well, note that these things seem to be the same (and therefore can't be causing the observed differences):

* The opponent is using the same evaluation function in all four cells
* The evaluation function used by the four test agents is exactly the same
* They're all using the same `AlphaBetaPlayer` code, and Alpha-Beta search isn't inherently a random algorithm
* The initial board configurations are exactly the same for each set of 50 games

The only differences I could think of were:

1. Differences in depth searched, due to differences in available computing resources. I started that tournament on an otherwise idling computer (with a quad-core processor) and then walked away from it while it ran, but maybe some background task was finishing up while the first test agent played, and then another background task started with the fourth agent was playing, meaning they could search less deeply in the provided time, resulting in their lower win rates.

2. The single piece of randomness in my Alpha-Beta implementation: the initialization of `best_move` to a random legal move before the search begins.

(This list actually used to have 3 items: I misread the code in `tournament.py` and thought that the test agents could be playing with different initial board configurations, which seemed like a huge potential source of variability in a given row. So I rewrote `tournament.py` to implement what I called "fair rounds," where the same opening moves are used for all test agents' games, only to learn that it already did that. Oops. I will note that the code can be made much cleaner by using `Counter`s to count things.)

Controlling for #1 would be doable: we could let the first test agent search as deeply as it could before time ran out, and then limit the rest of the test agents based on depth, not time. 

But controlling for #2 was laughably easy. While reading the project code, I noticed that `Board.__get_moves` first produces all legal moves in the same order every time, and then shuffles (randomizes) that list of moves before it returns it. So when I wrote my `AlphaBeta` code, I didn't bother to shuffle the result of `Board.get_legal_moves` (as many people in the forums, mentors included, were recommending), because it would just be a complication and a waste of computational resources - the list is already shuffled! Instead, I just initialized `best_move = game.get_legal_moves(self)[0]`, which gets a random legal move if `Board.__get_moves` is shuffling, and **which will return the same move every time, given the board configuration, if that "shuffle" line is commented out**. Commenting out that single line of code (`random.shuffle(valid_moves)` in `Board.__get_moves`) and rerunning the tournament produced the following results:

![Tournament, all test agents using ab_improved, NUM_MATCHES = 25. Same best_move initialization every time, given the board configuraton.](report-pics/section3/tourn2.png)

Note how much variability across the rows has decreased. The main exception to this is `Random`, but this particular opponent obviously has a source of randomness that the `MinimaxPlayer`s and `AlphaBetaPlayer`s don't.

**Q: Why does the best_move initialization matter so much? After all, I have this sweet search algorithm - shouldn't it almost immediately find a better move and overwrite that initial best_move value?**

Great question. In game situations where there are at least one decent legal move, the first decent move should result in a board with a much higher score than $-\infty$, causing `best_move` to be overwritten. But the way that I coded it, `best_move` is only overwritten if a legal move has a score *strictly greater* than the previous `best_move`. And if it's looking bleak for our test agent, in the sense that all roads appear to lead to a loss, then `best_move` will never actually get overwritten. 

This is the source of many of the forfeits mentioned in the forums: if you initialize `best_move` to `(-1, -1)` and the above effect occurs, then the agent will forfeit as soon as it seems like it's out of options. But immediately forfeiting instead of playing some legal move is a highly suboptimal strategy: this apparent bleakness is a function of the probably-not-perfect evaluation function that the agent is using, so it might not even be an accurate reflection of the situation. And even if it is accurate, playing a legal move gives the opponent an opportunity to forfeit, play a bad move, etc., which could salvage the game for our agent.

**Q: Is there a better way to initialize `best_move` than just picking a random legal move? It seems like it gets played a fair amount, so optimizing this could be worth it.**

Probably. For example, you could initialize it to the available move that moves you closest to the center of the board, or to the available move that achieves some other short-term goal. I didn't explore this idea, but it's a good one.

**Q: Does this decrease in variability hold when we reduce `NUM_MATCHES` back to something that's more computationally tractable?**

It certainly seems to:

![Tournament, all test agents using ab_improved, NUM_MATCHES = 5, non-random best_move initialization](report-pics/section3/num-matches-5/tourn1.png)

![Tournament, all test agents using ab_improved, NUM_MATCHES = 5, non-random best_move initialization](report-pics/section3/num-matches-5/tourn5.png)


**Q: Couldn't we make this even more computationally tractable by parallelizing the tournament? Evaluating multiple evaluation functions (in independent matches) seems like an embarassingly parallel problem.**

Yes, I think we can. I played around with this idea on my own a little bit before learning that there's a git branch dedicated to parallel tournaments. I started using that, and computational time decreased significantly, as expected.

**Q: So what's the plan moving forward?**

* The reduction in variation achieved by initializating `best_move` to the first available legal move consistently should make it much easier to visually attribute differences between Win Rate to the evaluation functions used and not to random variation
* To further reduce variation, let's **remove `Random` from the gauntlet of opponents**
* This reduction in variation seems to hold even when `NUM_MATCHES` is relatively small
* And running the tournaments in parallel makes things go faster

**So let's run a bunch of tournaments with `AB_Improved` and test agents equipped with different evaluation functions to see if we can find an evaluation function that outperforms `score_improved`!**

## 4: Infrastructure

As hinted at in the Introductory section of this report, the evaluation functions we try will be linear combinations of various features of the Isolation board:

In [6]:
### EVALUATION FUNCTION BUILDING BLOCKS: BOARD FEATURES ###
def num_moves(game, player):
    """How many legal moves does `player` have in `game`?"""
    return len(game.get_legal_moves(player))

def num_moves_opponent(game, player):
    """How many legal moves does the opponent of `player` have in `game`?"""
    return len(game.get_legal_moves(game.get_opponent(player)))

def dist_from_center(game, player):
    """What is `player`'s Manhattan distance from the center of the board?"""
    center_x = game.height / 2
    center_y = game.width / 2
    player_x, player_y = game.get_player_location(player)
    return abs(center_x - player_x) + abs(center_y - player_y)

def dist_from_opponent(game, player):
    """What is `player`'s Manhattan distance from its opponent?"""
    player_x, player_y = game.get_player_location(player)
    opponent = game.get_opponent(player)
    opp_x, opp_y = game.get_player_location(opponent)
    return abs(opp_x - player_x) + abs(opp_y - player_y)

HEURISTICS = (num_moves, num_moves_opponent, dist_from_center, dist_from_opponent)

Next, we'll need a way to generate tournaments Agents that use `AlphaBetaPlayer`s, equipped with linear combinations of these board features.

In [7]:
from collections import namedtuple
import random

from game_agent import AlphaBetaPlayer

WAgent = namedtuple("WAgent", ["player", "name", "weights"])   # tournament Agent with weights

def make_AB_WAgent(weights, fns=HEURISTICS):
    """Make a WAgent, using a linear combo of `weights` and `fns`."""
    score_fn = build_weighted_fn(weights, fns)
    name = weights_to_string(weights)
    return WAgent(AlphaBetaPlayer(score_fn=score_fn), name, weights)

# Helper functions
def build_weighted_fn(weights, fns):
    """
    Given weights and heuristic functions, create a score function that's a linear
    combination those weights and heuristics.
    """
    assert len(weights) == len(fns)
    def score_fn(game, player):
        if game.is_loser(player):
            return float("-inf")
        if game.is_winner(player):
            return float("inf")
        # skip features with weight == 0 to avoid computational time
        return sum(w * f(game, player) for w, f in zip(weights, fns) if w)
    return score_fn

def weights_to_string(weights):
    template = "|".join(["{}"] * len(weights))
    return template.format(*weights)

Which linear combinations should we consider? I think it makes sense to consider positive coefficients for `num_moves` and negative coefficients for `num_moves_opponent`. I'm not sure how `dist_from_center` and `dist_from_opponent` should be treated, so I'll consider both positive and negative coefficients, and I'll make their scales similar so that they can have an equally large impact on the board's score if they have something to say.

Here are the coefficient values I'll consider for each board feature:

In [8]:
from functools import reduce
from operator import mul

COEFFS = (
    (0, 1, 2, 3, 4),       # num_moves
    (-4, -3, -2, -1, 0),   # num_moves_opponent
    (-4, -2, 0, 2, 4),     # dist_from_center
    (-4, -2, 0, 2, 4)      # dist_from_opponent
)

# How many combinations?
reduce(mul, (len(cs) for cs in COEFFS), 1)

625

Let's build some agents using our coefficients:

In [9]:
from itertools import product

def make_all_agents():
    for weights in product(*COEFFS):
        yield make_AB_WAgent(weights)

And finally, we'll need to enter three of our agents at a time into a tournament with `AB_Improved` and capture the results of the tournament. (It would be straightforward to modify `tournament.py` to accept more than four test agents, but four test agents per tournament will be fine for our purposes.)

Unfortunately, `tournament.py` is geared towards printing the results to the terminal, not silently crunching and then saving the results. Fortunately, that's easy enough to implement:

In [14]:
from tournament import Agent
from game_agent import custom_score, custom_score_2, custom_score_3, MinimaxPlayer
from sample_players import open_move_score, center_score

test_agents = [
    Agent(AlphaBetaPlayer(score_fn=improved_score), "AB_Improved"),
    make_AB_WAgent((2, -1, 0, 0)),
    make_AB_WAgent((1, -2, 0, 0)),
    make_AB_WAgent((2, -3, 1, -1))
]

cpu_agents = [
    Agent(MinimaxPlayer(score_fn=open_move_score), "MM_Open"),
    Agent(MinimaxPlayer(score_fn=center_score), "MM_Center"),
    Agent(MinimaxPlayer(score_fn=improved_score), "MM_Improved"),
    Agent(AlphaBetaPlayer(score_fn=open_move_score), "AB_Open"),
    Agent(AlphaBetaPlayer(score_fn=center_score), "AB_Center"),
    Agent(AlphaBetaPlayer(score_fn=improved_score), "AB_Improved")
]

In [15]:
from collections import Counter

from tournament import play_round



def run_tournament(cpu_agents, test_agents, num_matches):
    """
    Play the same fair matches between the test agent and each cpu_agent individually.
    Returns the total number of games won by each test_agent
    Based on code provided by Udacity in tournament.py
    """
    total_wins = Counter()
    for agent in cpu_agents:
        wins = Counter()
        counts = play_round(agent, test_agents, wins, num_matches)
        del wins[agent]
        print(wins)
        total_wins.update(wins)
        
    print(total_wins)
    
    # The keys of total_wins are the test_agent.players
    # We'd like to group the weights with the number of wins
    abi, custom_agents = test_agents[0], test_agents[1:]
    abi_wins = total_wins[abi.player]
    weight_wins = dict()
    for ta in custom_agents:
        ta_wins = total_wins[ta.player]
        if ta_wins > abi_wins:
            weight_wins[ta.weights] = ta_wins
            
    return weight_wins

Let's see this is action:

In [16]:
run_tournament(cpu_agents, test_agents, 1)

AttributeError: Can't pickle local object 'build_weighted_fn.<locals>.score_fn'

Ouch, an `AttributeError` about pickling? A bit of Googling revealed that 